In [1]:
_RIOT_API_KEY='RGAPI-f133c0d1-25f4-4ab2-ad9e-7d96aa3b73f5'
from riotwatcher import LolWatcher
import helper_functions as help
import role_classification as rc
import constants
import pandas as pd
import numpy as np
from datetime import datetime
import mysql.connector
from sqlalchemy import create_engine
import concurrent.futures

from collections import Counter
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

from itertools import compress

lol_watcher = LolWatcher(_RIOT_API_KEY)

In [2]:
#match = lol_watcher.match.by_id('na1', 'NA1_3891774034')
#match = lol_watcher.match.by_id('na1', 'NA1_4720874733')
engine = help.create_mysql_engine()

# Plan for New Role Classification
1. Create new DB table 'lol_match_roles' with: matchId, participantId, Role, validRoles
2. Re-Create function for Jungle Flag
2. Re-Create function for Support Flag
3. Re-Create function for Lane By Posx/Posy
4. Create Mother Function 'determine_roles' To Validate positions for a match. validRoles = True if 1 player in each role, False otherwise. Upload to new 
5. Probably create a 'lol_analysis_view' which pulls needed info from 'lol_match_details', 'lol_match_timeline', and 'lol_match_roles' which an analysis script can pull from.


In [4]:
minutes_to_analyse = [10,14]
summ_name = 'souporsecret'
query=f"""      
SELECT
    det.matchId,
    det.championName,
    det.summonerName,
    rol.role,
    det.winFlag,
    det.teamName,
    case when summoner_basis.teamName = det.teamName then True else False end as teamMate,
    det.queueName,
    det.gameCreationDate,
    time.minute,
    time.participantId,
    time.xp,
    time.minionsKilled,
    time.jungleMinionsKilled,
    time.minionsKilled + time.jungleMinionsKilled as totalCS,
    time.totalDamageDoneToChampions
FROM riot_api.lol_match_timeline time
INNER JOIN riot_api.lol_match_details det on time.matchId=det.matchId and time.participantId=det.participantId
INNER JOIN riot_api.lol_match_roles rol on rol.matchId=det.matchId and rol.participantId=det.participantId
INNER JOIN (select distinct matchid, teamName FROM riot_api.lol_match_details where summonerName = ''{summ_name}'') summoner_basis
			on det.matchid = summoner_basis.matchid
where rol.validRole = 1 and minute in {tuple(minutes_to_analyse)}
	  and det.matchid in (select distinct matchid FROM riot_api.lol_match_details where summonerName = '{summ_name}')
;

 """
df = pd.read_sql(query, con=engine)
df

,matchId,championName,summonerName,role,winFlag,teamName,teammate,queueName,gameCreationDate,minute,participantId,xp,minionsKilled,jungleMinionsKilled,totalCS,totalDamageDoneToChampions
0,NA1_4720874733,Thresh,Sunamii,SUPPORT,0,Red,0,5v5 Ranked Flex games,2023-07-21 22:41:19,14,10,2803,26,0,26,1925
1,NA1_4720874733,Kaisa,Tian Qing Sè,BOTTOM,0,Red,0,5v5 Ranked Flex games,2023-07-21 22:41:19,14,9,5475,101,0,101,6757
2,NA1_4720874733,Lux,billionfly,MIDDLE,0,Red,0,5v5 Ranked Flex games,2023-07-21 22:41:19,14,8,6986,98,0,98,5526
3,NA1_4720874733,LeeSin,KingOliveOil,JUNGLE,0,Red,0,5v5 Ranked Flex games,2023-07-21 22:41:19,14,7,4806,0,78,78,2206
4,NA1_4720874733,Mordekaiser,IDKMM,TOP,0,Red,0,5v5 Ranked Flex games,2023-07-21 22:41:19,14,6,6411,76,0,76,5399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15105,NA1_3834718751,Lulu,Bread303,SUPPORT,0,Blue,0,Clash games,2021-03-20 19:01:19,10,1,2562,0,0,0,871
15106,NA1_3834718751,Mordekaiser,radaemon1,TOP,1,Red,1,Clash games,2021-03-20 19:01:19,14,6,6894,108,0,108,2949
15107,NA1_3834718751,Mordekaiser,radaemon1,TOP,1,Red,1,Clash games,2021-03-20 19:01:19,10,6,4435,70,0,70,1398
15108,NA1_3834718751,Viego,satanwilrize,JUNGLE,0,Blue,0,Clash games,2021-03-20 19:01:19,14,4,5706,10,96,106,1365
